# Yandex Market Image Processor - Демонстрация

Этот notebook демонстрирует основные возможности сервиса обработки изображений для Яндекс Маркет.

## 1. Импорт необходимых библиотек

In [ ]:
import sys
from pathlib import Path

# Добавляем путь к src в PYTHONPATH
sys.path.append(str(Path('..').resolve()))

import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

from src.processors.background import BackgroundRemover
from src.utils.image_helpers import load_image, save_image, calculate_image_complexity

## 2. Создание тестовых изображений

In [ ]:
def create_test_image(type='simple'):
    """Создает тестовое изображение для демонстрации."""
    
    if type == 'simple':
        # Простое изображение: красный квадрат на сером фоне
        img = Image.new('RGB', (400, 400), color=(200, 200, 200))
        draw = ImageDraw.Draw(img)
        draw.rectangle([100, 100, 300, 300], fill=(255, 0, 0))
        
    elif type == 'complex':
        # Сложное изображение: несколько объектов на градиентном фоне
        img = Image.new('RGB', (400, 400))
        pixels = img.load()
        
        # Градиентный фон
        for i in range(400):
            for j in range(400):
                pixels[i, j] = (i * 255 // 400, j * 255 // 400, 200)
        
        draw = ImageDraw.Draw(img)
        # Основной объект
        draw.ellipse([150, 150, 250, 250], fill=(0, 255, 0))
        # Дополнительные детали
        draw.rectangle([180, 180, 220, 220], fill=(255, 255, 0))
        
    elif type == 'realistic':
        # Более реалистичное изображение товара
        img = Image.new('RGB', (400, 400), color=(240, 240, 240))
        draw = ImageDraw.Draw(img)
        
        # "Товар" - стилизованная коробка
        # Передняя грань
        draw.polygon([(100, 200), (200, 150), (200, 350), (100, 300)], 
                    fill=(100, 150, 200), outline=(50, 75, 100))
        # Верхняя грань
        draw.polygon([(100, 200), (200, 150), (300, 200), (200, 250)], 
                    fill=(120, 170, 220), outline=(50, 75, 100))
        # Боковая грань
        draw.polygon([(200, 150), (300, 200), (300, 300), (200, 350)], 
                    fill=(80, 130, 180), outline=(50, 75, 100))
    
    return img

# Создаем тестовые изображения
simple_img = create_test_image('simple')
complex_img = create_test_image('complex')
realistic_img = create_test_image('realistic')

# Отображаем
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(simple_img)
axes[0].set_title('Простое изображение')
axes[0].axis('off')

axes[1].imshow(complex_img)
axes[1].set_title('Сложное изображение')
axes[1].axis('off')

axes[2].imshow(realistic_img)
axes[2].set_title('Реалистичное изображение')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 3. Инициализация процессора удаления фона

In [ ]:
# Создаем процессор с базовыми настройками
remover = BackgroundRemover({
    'debug': True,
    'post_process': True,
    'min_object_size': 500
})

print(f"Процессор инициализирован: {remover.name}")
print(f"Модель: {remover.model_name}")
print(f"Пост-обработка: {'включена' if remover.post_process else 'выключена'}")

## 4. Удаление фона - простой пример

In [ ]:
# Обрабатываем простое изображение
result_simple, mask_simple = remover.process(simple_img, return_mask=True)

# Визуализация результатов
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(simple_img)
axes[0].set_title('Исходное изображение')
axes[0].axis('off')

axes[1].imshow(mask_simple, cmap='gray')
axes[1].set_title('Маска')
axes[1].axis('off')

axes[2].imshow(result_simple)
axes[2].set_title('Результат (прозрачный фон)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 5. Обработка сложного изображения

In [ ]:
# Анализируем сложность изображения
complexity = calculate_image_complexity(complex_img)
print("Метрики сложности:")
for key, value in complexity.items():
    print(f"  {key}: {value:.3f}")

# Обрабатываем
result_complex, mask_complex = remover.process(complex_img, return_mask=True)

# Визуализация
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

axes[0, 0].imshow(complex_img)
axes[0, 0].set_title('Исходное изображение')
axes[0, 0].axis('off')

axes[0, 1].imshow(mask_complex, cmap='gray')
axes[0, 1].set_title('Маска')
axes[0, 1].axis('off')

axes[1, 0].imshow(result_complex)
axes[1, 0].set_title('Результат')
axes[1, 0].axis('off')

# Результат на цветном фоне для демонстрации
demo_bg = Image.new('RGB', result_complex.size, color=(255, 200, 200))
demo_bg.paste(result_complex, (0, 0), result_complex)
axes[1, 1].imshow(demo_bg)
axes[1, 1].set_title('На цветном фоне')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 6. Сравнение с и без пост-обработки

In [ ]:
# Создаем два процессора
remover_no_post = BackgroundRemover({'post_process': False})
remover_with_post = BackgroundRemover({'post_process': True})

# Обрабатываем реалистичное изображение
result_no_post, mask_no_post = remover_no_post.process(realistic_img, return_mask=True)
result_with_post, mask_with_post = remover_with_post.process(realistic_img, return_mask=True)

# Визуализация
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Без пост-обработки
axes[0, 0].imshow(realistic_img)
axes[0, 0].set_title('Исходное')
axes[0, 0].axis('off')

axes[0, 1].imshow(mask_no_post, cmap='gray')
axes[0, 1].set_title('Маска без пост-обработки')
axes[0, 1].axis('off')

axes[0, 2].imshow(result_no_post)
axes[0, 2].set_title('Результат без пост-обработки')
axes[0, 2].axis('off')

# С пост-обработкой
axes[1, 0].imshow(realistic_img)
axes[1, 0].set_title('Исходное')
axes[1, 0].axis('off')

axes[1, 1].imshow(mask_with_post, cmap='gray')
axes[1, 1].set_title('Маска с пост-обработкой')
axes[1, 1].axis('off')

axes[1, 2].imshow(result_with_post)
axes[1, 2].set_title('Результат с пост-обработкой')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## 7. Пакетная обработка

In [ ]:
# Создаем несколько тестовых изображений
test_images = [
    create_test_image('simple'),
    create_test_image('complex'),
    create_test_image('realistic')
]

# Сохраняем во временную директорию
import tempfile
import os

with tempfile.TemporaryDirectory() as temp_dir:
    # Сохраняем изображения
    image_paths = []
    for i, img in enumerate(test_images):
        path = os.path.join(temp_dir, f'test_{i}.png')
        img.save(path)
        image_paths.append(path)
    
    # Пакетная обработка
    def progress_callback(current, total):
        print(f"Обработано {current}/{total} изображений")
    
    results = remover.process_batch(
        image_paths, 
        output_dir=temp_dir,
        progress_callback=progress_callback
    )
    
    # Показываем результаты
    fig, axes = plt.subplots(len(results), 2, figsize=(8, 12))
    
    for i, result in enumerate(results):
        if result['success']:
            # Исходное
            axes[i, 0].imshow(test_images[i])
            axes[i, 0].set_title(f'Исходное {i+1}')
            axes[i, 0].axis('off')
            
            # Результат
            axes[i, 1].imshow(result['image'])
            axes[i, 1].set_title(f'Результат {i+1}')
            axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

## 8. Настройки для разных типов товаров

In [ ]:
# Примеры конфигураций для разных категорий
configs = {
    'fashion': {
        'alpha_matting': True,  # Для мягких краев одежды
        'post_process': True,
        'min_object_size': 2000  # Одежда обычно крупная
    },
    'beauty': {
        'alpha_matting': True,  # Для прозрачных флаконов
        'post_process': True,
        'min_object_size': 500  # Косметика может быть мелкой
    },
    'electronics': {
        'alpha_matting': False,  # Четкие края у техники
        'post_process': True,
        'min_object_size': 1000
    }
}

# Демонстрация с разными настройками
test_img = create_test_image('realistic')

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_img)
axes[0].set_title('Исходное')
axes[0].axis('off')

for i, (category, config) in enumerate(configs.items()):
    remover_cat = BackgroundRemover(config)
    result = remover_cat.process(test_img)
    
    axes[i+1].imshow(result)
    axes[i+1].set_title(f'Настройки: {category}')
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

## 9. Сохранение результатов

In [ ]:
# Пример сохранения обработанного изображения
from src.utils.image_helpers import save_image

# Обрабатываем изображение
final_result = remover.process(realistic_img)

# Сохраняем в разных форматах
output_dir = Path('../data/output')
output_dir.mkdir(exist_ok=True)

# PNG с прозрачностью
png_path = save_image(
    final_result,
    output_dir / 'demo_result.png',
    format='PNG'
)
print(f"Сохранено как PNG: {png_path}")

# Для демонстрации - на белом фоне
white_bg = Image.new('RGB', final_result.size, color=(255, 255, 255))
white_bg.paste(final_result, (0, 0), final_result)

jpg_path = save_image(
    white_bg,
    output_dir / 'demo_result_white_bg.jpg',
    format='JPEG',
    quality=95
)
print(f"Сохранено как JPEG: {jpg_path}")

## 10. Производительность и оптимизация

In [ ]:
import time

# Тестируем производительность
sizes = [(200, 200), (400, 400), (800, 800), (1600, 1600)]
times = []

for size in sizes:
    # Создаем тестовое изображение
    test_img = Image.new('RGB', size, color=(200, 200, 200))
    draw = ImageDraw.Draw(test_img)
    draw.ellipse([size[0]//4, size[1]//4, 3*size[0]//4, 3*size[1]//4], 
                 fill=(100, 150, 200))
    
    # Измеряем время
    start_time = time.time()
    result = remover.process(test_img)
    end_time = time.time()
    
    processing_time = end_time - start_time
    times.append(processing_time)
    
    print(f"Размер {size[0]}x{size[1]}: {processing_time:.2f} сек")

# График производительности
plt.figure(figsize=(10, 6))
plt.plot([s[0] for s in sizes], times, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Размер изображения (пиксели)')
plt.ylabel('Время обработки (сек)')
plt.title('Зависимость времени обработки от размера изображения')
plt.grid(True, alpha=0.3)
plt.show()

## Заключение

В этом notebook мы продемонстрировали:

1. **Базовое использование** модуля удаления фона
2. **Обработку различных типов изображений** - от простых до сложных
3. **Важность пост-обработки** для улучшения качества масок
4. **Пакетную обработку** нескольких изображений
5. **Настройки для разных категорий** товаров
6. **Сохранение результатов** в различных форматах
7. **Анализ производительности** в зависимости от размера

Следующие шаги:
- Реализация модулей для генерации теней
- Добавление цветокоррекции
- Интеграция с AI сервисами для upscaling
- Создание полного pipeline обработки